In [2]:
# Importing the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns



## READING THE DATA

In [79]:
books = pd.read_csv("./Data/Books.csv", sep = ",",on_bad_lines='skip', encoding='latin-1')

/var/folders/qj/rbyl_gdj7xn3ycgcxjpvn5080000gn/T/ipykernel_8866/2055226375.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("./Data/Books.csv", sep = ",",on_bad_lines='skip', encoding='latin-1')


In [80]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [81]:
# Checking number of records
books.shape

(271360, 8)

In [82]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [83]:
# Dropping columns

books.drop(columns=['Image-URL-S','Image-URL-M'],inplace=True)

In [84]:
# Renaming columns
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Pbulication':'year','Publisher':'publisher','Image-URL-L':'img_url'}, inplace=True)

In [85]:
# Checking users
users = pd.read_csv("./Data/Users.csv", on_bad_lines='skip', encoding= 'latin-1')

In [86]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [87]:
# Checking users
ratings = pd.read_csv("./Data/Ratings.csv", on_bad_lines='skip', encoding= 'latin-1')

In [88]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [89]:
ratings.rename(columns={'User-ID':'user_id','Book-Rating':'rating'}, inplace=True)

In [90]:
ratings

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [91]:
x = ratings['user_id'].value_counts() > 200

In [92]:
y = x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
        88793,  44296, 268622, 155916,  73681, 274808,  28634, 188951,  59727,
         9856],
      dtype='int64', name='user_id', length=899)

In [93]:
ratings_filtered = ratings[ratings['user_id'].isin(y)]

In [94]:
ratings_with_books = ratings_filtered.merge(books, on='ISBN')

In [95]:
ratings_with_books.shape

(487671, 8)

In [96]:
num_rating =ratings_with_books.groupby('title')['rating'].count().reset_index()

In [97]:
num_rating.rename(columns={"rating" : "num_of_rating"},inplace=True)

In [98]:
final_rating= ratings_with_books.merge(num_rating,on='title')

In [102]:
final_rating = final_rating[final_rating['num_of_rating']>=50]

In [103]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [104]:
final_rating.shape

(59850, 9)

In [106]:
books_pivot = pd.pivot_table(columns=['user_id'],index=['title'],values=['rating'],data= final_rating)

In [107]:
books_pivot.shape

(742, 888)

In [113]:
books_pivot.fillna(value=0, inplace=True)

In [114]:
from scipy.sparse import csr_matrix

In [115]:
book_sparse = csr_matrix(books_pivot)

In [116]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [117]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [118]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [119]:
distance, suggestion = model.kneighbors(books_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [120]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [130]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [140]:
for i in range(len(suggestion)):
    print(books_pivot.iloc[suggestion[i]].index)

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [141]:
books_name = books_pivot.index


In [149]:
import pickle
import os
os.makedirs('artifacts',exist_ok=True)
pickle.dump(model,open('./artifacts/model.pkl','wb'))
pickle.dump(books_name,open('./artifacts/books_name.pkl','wb'))
pickle.dump(final_rating,open('./artifacts/final_rating.pkl','wb'))
pickle.dump(books_pivot,open('./artifacts/book_pivot.pkl','wb'))


In [152]:
def recommend_book(book_name):
    book_id = np.where(books_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(books_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

    for i in range(len(suggestion)):
        books = books_pivot.iloc[suggestion[i]].index
        for j in books:
            print(j)


In [153]:
book_name= "The Cradle Will Fall"
recommend_book(book_name)

The Cradle Will Fall
Exclusive
The Long Road Home
Eyes of a Child
Jacob Have I Loved
No Safe Place


In [154]:
final_rating


,user_id,ISBN,rating,title,author,Year-Of-Publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50


In [159]:
arr= (1,2,3)

In [160]:
type(arr)

tuple